In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

import warnings
from copy import deepcopy
import gc
from collections import Counter

%matplotlib inline

In [17]:
def score_model(model, X, y, threshold=0.5):  # 0.41 is the best for now
    pred = model.predict_proba(X)[:, 1]
    roc_auc = roc_auc_score(y, pred, multi_class='ovo')
    return roc_auc

In [2]:
%cd ../

/home/chervovn04/Programming/hackathons/2022/digital_breakout_885303


In [3]:
cat_features = ["type", "is_privatecategory", "class", "is_in_yandex", "mailctg", "directctg", "mailtype"]

def to_cat_features(df):
    for cat_feature in cat_features:
        df[cat_feature] = df[cat_feature].apply(lambda text: str(text))
        
X_train = pd.read_csv('data/X_train_prepared.csv')
# y_train = pd.read_csv('data/y_train_prepared.csv')
X_val = pd.read_csv('data/X_val_prepared.csv')
# y_val = pd.read_csv('data/y_val_prepared.csv')
test_df = pd.read_csv('data/test_df_prepared.csv')
# test_ids = pd.read_csv('data/test_ids.csv')

to_cat_features(X_train)
to_cat_features(X_val)
to_cat_features(test_df)

In [5]:
X = pd.concat([X_train, X_val, test_df], ignore_index=True) 
y = pd.concat([pd.Series(np.zeros(X_train.shape[0] + X_val.shape[0])), pd.Series(np.ones(test_df.shape[0]))], ignore_index=True)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=0)

In [13]:
model = CatBoostClassifier(random_state=0, max_depth=8, verbose=10, iterations=30, auto_class_weights="Balanced", cat_features=cat_features)

In [14]:
model.fit(X_train, y_train) 

/home/chervovn04/Programming/hackathons/2022/digital_breakout_885303/venv/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


Learning rate set to 0.5
0:	learn: 0.6931290	total: 757ms	remaining: 22s
10:	learn: 0.6929482	total: 7.8s	remaining: 13.5s
20:	learn: 0.6927434	total: 15.2s	remaining: 6.5s
29:	learn: 0.6926350	total: 21.2s	remaining: 0us


In [18]:
score_model(model, X_test, y_test)

/home/chervovn04/Programming/hackathons/2022/digital_breakout_885303/venv/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


0.5001453451669878

In [19]:
model.feature_importances_

array([5.31215684, 1.71392605, 0.65280585, 8.36782739, 2.60431636,
       1.38393653, 7.06213476, 4.98900022, 8.31521863, 0.21213483,
       5.94139717, 0.74605561, 7.44787547, 9.47972979, 4.0801888 ,
       6.03504583, 3.29165635, 4.27592797, 6.39309707, 0.85276171,
       1.30245992, 2.30378267, 0.54300405, 0.04719065, 0.38540317,
       0.16534786, 0.        , 0.        , 0.        , 0.        ,
       0.03832471, 0.        , 0.26770384, 5.78958991])